In [1]:
!pip install -q streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [91]:
app_code = app_code = """
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel, PeftConfig
import torch

st.set_page_config(page_title="💬 Fine-Tuned TinyLLaMA LLM Chat", layout="wide")
st.title("💬 Fine-Tuned TinyLLaMA LLM Chat")

# Path to your unzipped LoRA adapter
ADAPTER_PATH = "/content/dpo-trial3/dpo-trial3"
#ADAPTER_PATH = "/content/sft-trial4/sft-trial4"

@st.cache_resource
def load_model():
    # Load adapter config
    config = PeftConfig.from_pretrained(ADAPTER_PATH)

    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

    # Apply LoRA adapter
    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

    # Create text generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=False,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        return_full_text=True,
        eos_token_id=tokenizer.eos_token_id
    )

    return pipe, tokenizer

generator, tokenizer = load_model()

prompt = st.text_area("Enter your prompt:", height=150)

if st.button("Generate Response"):
    with st.spinner("Generating..."):
        try:
            # Format prompt
            formatted_prompt = f"### Instruction:\\n{prompt}\\n\\n### Response:"
            output = generator(formatted_prompt)[0]["generated_text"]
            response_only = output.split("### Response:")[-1].strip()

            # Clean output (optional)
            def clean_output(text):
                return text.split("###")[0].split("\\n")[0].strip()

            st.markdown("**Response:**")
            st.markdown(response_only.replace("\\n", "  \\n"))

            st.write(clean_output(response_only))

        except Exception as e:
            st.error(f"Generation error: {str(e)}")
"""


with open("app.py", "w") as f:
    f.write(app_code)


In [ ]:
#!unzip -o dpo-trial3_safe.zip -d /content/dpo-trial3

In [92]:
!streamlit run app.py &>/content/log.txt &

In [8]:
import requests
print("Tunnel password (public IP):", requests.get("https://loca.lt/mytunnelpassword").text)

Tunnel password (public IP): 34.21.93.6


In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦your url is: https://cool-wombats-knock.loca.lt
